In [2]:
import logging
import time

import glob
import pandas as pd

from typing import List, Dict, Any, Union

# Импорт утилитарных функций для работы с данными: 
# проверка пропусков, сохранение и открытие JSON файлов
from data_utils import check_missing, save_json, open_json

# Функции для парсинга
from api_fetcher import split_list, APIFetcher

# Словари параметров для API запросов к Кинопоиску
from config_params import params_movies_main, params_movies_additional, params_persons

# Функции для подготовки данных из API к DataFrame
from data_transformers import get_movies, make_persons_movies_dict

logging.basicConfig(level=logging.INFO)

with open('api_token.txt') as file:
    API_TOKEN = file.read().strip()

API_URL_MOVIES = 'https://api.kinopoisk.dev/v1.3/movie'
API_URL_PERSON = 'https://api.kinopoisk.dev/v1/person'
API_URL_STUDIOS = 'https://api.kinopoisk.dev/v1/studio'
HEADERS = {'X-API-KEY': API_TOKEN}

# Параметры для сохранения csv файлов
SAVE_DICT = {'sep': ';', 'index': False}

# Описание задачи

**Цель проекта:** Спарсить данные о фильмах с сайта Кинопоиск для последующего обучения модели машинного обучения, которая будет предсказывать сборы фильма в США.

**Цель текущего ноутбука:** Извлечь данные с сайта Кинопоиск о фильмах, студиях и участниках кинопроектов, а также подготовить необходимую информацию для последующего расчета метрик персон.

**Порядок парсинга:**

**1. Поиск фильмов на Кинопоиске по фильтрам:**
   - Год выпуска фильма: 2000 - 2023.
   - Тип картины: фильм или мультфильм.
   - Присутствует информация о сборах в США.
   - Количество оценок: не менее 2000.

**2. Сбор информации о студиях:**
   - Найти информацию о студиях каждого фильма из пункта 1.

**3. Сбор базовой информации о персонах:**
   - Найти информацию о всех актёрах, режиссёрах и сценаристах, связанных с фильмами из пункта 1.

**4. Детальный сбор данных о фильмах персон:**
   - Извлечь полную информацию о фильмах, в которых участвовали персоны из пункта 3. Это необходимо для дальнейшего расчета метрик в ноутбуке "3.Метрики персон".

In [2]:
# Создаем объект класса для парсинга
fetcher = APIFetcher(headers=HEADERS)

# Поиск фильмов на Кинопоиске

In [25]:
# Инициализация параметров запроса для 1й страницы и загрузка первичных данных о фильмах.
params_movies_main['page'] = '1'
dict_data = {'url': API_URL_MOVIES, 'params': params_movies_main}

# Загрузка первой страницы
first_page_data = fetcher.fetch_data(**dict_data)

total_pages = first_page_data['pages']
json_movies_main = first_page_data['docs']

# Загрузка данных с последующих страниц 
for page in range(2, total_pages+1):

    dict_data['params']['page'] = str(page)
    logging.info(f'Загрузка страницы {page} из {total_pages}...')
    json_movies_main += fetcher.fetch_data(**dict_data)['docs']

INFO:root:Загрузка страницы 2 из 33...
INFO:root:Загрузка страницы 3 из 33...
INFO:root:Загрузка страницы 4 из 33...
INFO:root:Загрузка страницы 5 из 33...
INFO:root:Загрузка страницы 6 из 33...
INFO:root:Загрузка страницы 7 из 33...
INFO:root:Загрузка страницы 8 из 33...
INFO:root:Загрузка страницы 9 из 33...
INFO:root:Загрузка страницы 10 из 33...
INFO:root:Загрузка страницы 11 из 33...
INFO:root:Загрузка страницы 12 из 33...
INFO:root:Загрузка страницы 13 из 33...
INFO:root:Загрузка страницы 14 из 33...
INFO:root:Загрузка страницы 15 из 33...
INFO:root:Загрузка страницы 16 из 33...
INFO:root:Загрузка страницы 17 из 33...
INFO:root:Загрузка страницы 18 из 33...
INFO:root:Загрузка страницы 19 из 33...
INFO:root:Загрузка страницы 20 из 33...
INFO:root:Загрузка страницы 21 из 33...
INFO:root:Загрузка страницы 22 из 33...
INFO:root:Загрузка страницы 23 из 33...
INFO:root:Загрузка страницы 24 из 33...
INFO:root:Загрузка страницы 25 из 33...
INFO:root:Загрузка страницы 26 из 33...
INFO:roo

In [28]:
save_json('data_json/movies_main.json', json_movies_main)

In [17]:
# Преобразование данных о фильмах в DataFrame.
df = pd.DataFrame(get_movies(json_movies_main))

# Удаление строк, где отсутствуют данные о сборах фильма в США.
df = df[df.fees_usa!='']
df = df[~df.fees_usa.isna()]

print(df.shape)
df[:2]

(5329, 44)


,movie_id,movie_name,year,votes_kp,votes_imdb,votes_filmCritics,votes_await,rating_kp,rating_imdb,rating_filmCritics,...,fees_world,fees_world_currency,fees_usa,fees_usa_currency,fees_russia,fees_russia_currency,budget,budget_currency,videos_trailers_number,videos_teasers_number
0,535341,1+1,2011,1678838,895482,130,15,8.813,8.5,6.8,...,426588510,$,10198820,$,1725813,$,9500000,€,2.0,0.0
1,1143242,Джентльмены,2019,1524128,372380,277,13642,8.562,7.8,6.5,...,115171795,$,36471795,$,18003343,$,22000000,$,4.0,0.0


Описание колонок DataFrame с фильмами:

- **movie_id** - уникальный идентификатор фильма.
- **movie_name** - название фильма.
- **year** - год выпуска фильма.
- **votes_kp, votes_imdb** - количество оценок фильма на Кинопоиске и IMDB.
- **rating_kp, rating_imdb** - рейтинги фильма на Кинопоиске и IMDB.
- **votes_filmCritics** - количество оценок кинокритиков в мире
- **rating_filmCritics** - рейтинг кинокритиков в мире
- **votes_await** - количество ожидающих фильм на Кинопоиске
- **movieLength** - длина фильма
- **ageRating, ratingMpaa** - возрастные рейтинги фильма
- **type** - тип картины (фильм/мультфильм)
- **genres** - жанры фильма
- **countries** - страны фильма
- **художники, редакторы, актеры и т.д.** - имена персон, учавствующих в фильме
- **художники_id, редакторы_id, актеры_id и т.д.** - id персон, учавствующих в фильме
- **fees_world, fees_usa, fees_russia** - сборы фильма в мире, США, России
- **fees_world_currency и т.д.** - валюта сборов фильма
- **budget, budget_currency** - бюджет и валюта бюджета фильма
-**videos_trailers_number** - количество трейлеров
- **videos_teasers_number** - количество тизеров

In [18]:
df.to_csv('data/movies_main.csv', **SAVE_DICT)

# устанавливаем индекс по id для быстрого доступа
# чтобы в дальнейшем добавить имя фильма в таблицы для наглядности
df = df.set_index('movie_id')  

# id всех собранных фильмов - для парсинга студий
ids_movies_main = list(df.index.values.astype(str))  

# Сбор информации о студиях

In [ ]:
# Параметры запроса для извлечения данных о студиях фильмов.
params = {
    'selectFields': ['title', 'type'],
    'limit': '100',
    'page': '1'
} 

# Делим список всех id фильмов на части.
# Каждая часть содержит 50 id для более эффективного запроса данных о студиях.
ids = split_list(ids_movies_main, 50)

for i, chunk in enumerate(ids):
    """
    API возвращает детали о студиях только для одного фильма за раз.
    Для каждого id фильма из чанка (50 фильмов) отправляем индивидуальный запрос к API.
    """
    studios_chunk = []
    
    for movie_id in chunk:
        params['movies.id'] = movie_id
        studios_data = fetcher.fetch_data(API_URL_STUDIOS, params)['docs']
        studios_chunk.append((movie_id, studios_data))

    # Сохраняем данные о студиях для текущего чанка в JSON-файл.
    # Чтобы в случае ошибки или прерывания соединения, данные не потерялись.
    save_json(f'data_json/studios_PARTS/studios_chunk_{i}.json', studios_chunk)

    # Пауза между итерациями, чтобы не перегружать API и избегать блокировок или ошибок.
    time.sleep(30)

# После загрузки всех чанков, объединяем данные из всех JSON-файлов в один общий список.
path = 'data_json/studios_PARTS/*.json'
files = glob.glob(path)

# Объединяем данные в один общий список.
studios = [entry for file in files for entry in open_json(file)]

In [14]:
save_json('data_json/studios.json', studios)

In [19]:
# Создаем список словарей для DataFrame.
# Для каждого фильма в списке `studios` извлекаем его id и детали студий.
# Если для фильма нет деталей студии (studios пуст), используем пустой словарь.
df_data = [
    {'movie_id': film_id, 
     'movie_name': df.loc[int(film_id), 'movie_name'], 
     'title': studio.get('title'), 
     'type': studio.get('type')}
    for film_id, movie_studios in studios
    for studio in (movie_studios if studios else [{}])
]

studios_df = pd.DataFrame(df_data)
studios_df

,movie_id,movie_name,title,type
0,258536,Мать слёз,Myriad Pictures,Прокат
1,258536,Мать слёз,Apocalypsis,Спецэффекты
2,258536,Мать слёз,Myriad Pictures,Производство
3,258536,Мать слёз,Opera Film (II),Производство
4,258536,Мать слёз,Mitropoulos Films,Прокат
...,...,...,...,...
51523,77538,Синоптик,Escape Artists,Производство
51524,77538,Синоптик,Method Studios,Спецэффекты
51525,77538,Синоптик,Pacific Title & Art Studio,Спецэффекты
51526,77538,Синоптик,Paramount,Прокат


Каждая строка представляет собой уникальную связь фильма со студией.

Описание колонок DataFrame студий:

- **movie_id**: Идентификатор фильма.
- **movie_name**: Название фильма.
- **title**: Название студии.
- **type**: Роль студии (например, "Производство", "Прокат", "Спецэффекты" и т. д.).

In [6]:
studios_df.to_csv('data/studios.csv', sep=';', index=False)

# Сброр данных для метрик персон

1. **Первый этап:** 
<br>На этом этапе мы собираем информацию о различных персонах и фильмах, в которых они участвовали. Цель - получить базовые данные о каждой персоне и определить, в каких фильмах они участвовали, а также их роль в этих фильмах (например, актёр, режиссёр и т.д.).

   **Результат:** DataFrame, который содержит следующие колонки:
   
   - **person_id, person_name, age** и т.д.: Данные о персоне.
   - **movie_id, movie_name, movie_rating, movie_profession**: Информация о фильмах, в которых участвовала эта персона.


2. **Второй этап:** 
<br>Поскольку в первом DataFrame нет подробной информации о каждом фильме (как например, сборы, бюджет, год выпуска и т.д.), мы проводим дополнительный парсинг для получения детальной информации о каждом фильме.

   **Результат:** DataFrame, который содержит детальную информацию о каждом фильме, включая идентификаторы фильмов, названия, рейтинги, сборы и многое другое.

## Информация о персонах

In [20]:
def persons_ids_list(data, col):
    """
    Извлекает и возвращает уникальные идентификаторы (id) из указанной колонки DataFrame.
    
    Параметры:
    - data (DataFrame): DataFrame, из которого необходимо извлечь идентификаторы.
    - col (str): Название колонки, из которой извлекаются идентификаторы.
    
    Возвращает:
    - List[str]: Список уникальных идентификаторов.
    """
    
    # Извлекаем идентификаторы, игнорируя отсутствующие значения
    df_ids = data[[col]].explode(col).dropna()
    
    # Получаем уникальные идентификаторы и преобразуем их в строки
    ids_list = list(set(df_ids[col].astype(str)))
    
    print(f'Всего уникальных id в колонке {col}: {len(ids_list)}')
    return ids_list
    
ids_director = persons_ids_list(df, 'режиссеры_id')
# Редакторы здесь - это сценаристы на странице фильма на Кинопоиске
ids_editors = persons_ids_list(df, 'редакторы_id')
ids_actors = persons_ids_list(df, 'актеры_id')

Всего уникальных id в колонке режиссеры_id: 2820
Всего уникальных id в колонке редакторы_id: 6725
Всего уникальных id в колонке актеры_id: 23903


In [21]:
save_json('data_json/ids_directors.json', ids_director)
save_json('data_json/ids_editors.json', ids_editors)
save_json('data_json/ids_actors.json', ids_actors)

In [54]:
ids_all_persons = list(set(ids_director + ids_editors + ids_actors))
print('Всего уникальных id персон:', len(ids_all_persons))

Всего уникальных id персон: 31064


In [ ]:
# Делим список id персон на части для оптимизации количества запросов к API.
# Каждая часть содержит 200 id, так как API ограничивает количество id,
# которые можно использовать за один запрос.
ids = split_list(ids_persons, 200)

for i, chunk in enumerate(ids):
    """
    Для каждого чанка из 200 id персон отправляем запросы к API.
    Получаем информацию о каждой персоне и их фильмах.
    """
    
    json_info = fetcher.get_json_docs(
        API_URL_PERSON, params_persons, 'id', chunk, 200)

    # Сохраняем данные для текущего чанка в отдельный JSON-файл.
    save_json(f'data_json/one_person_one_movie_PARTS/json_ids_{i}', json_info)

    # Даём паузу между итерациями, чтобы не перегружать сервер и избежать блокировок.
    time.sleep(50)

# Объединяем все JSON-файлы в один общий список.
path = 'data_json/one_person_one_movie_PARTS/*.json'
files = glob.glob(path)
json_one_person_one_movie = [open_json(file) for file in files]

In [14]:
save_json('data_json/one_person_one_movie.json', json_one_person_one_movie)

In [16]:
df_one_person_one_movie = pd.DataFrame(
    make_persons_movies_dict(json_one_person_one_movie))

# Оставим фильмы с формировавшимся рейтингом кинопоиска
df_one_person_one_movie = df_one_person_one_movie[~df_one_person_one_movie.
                                                  movie_rating.isna()].copy()
# Оставим фильмы где нет порпуска в профессии
df_one_person_one_movie = df_one_person_one_movie[
    ~df_one_person_one_movie.movie_profession.isna()].copy()

df_one_person_one_movie.drop_duplicates(inplace=True)
print(df_one_person_one_movie.shape)
df_one_person_one_movie[:5]

(679539, 9)


,person_id,person_name,age,countAwards,death,movie_id,movie_name,movie_rating,movie_profession
2,6966712,Хан Мин-ёп,33.0,NaN,None,4446955.0,Децибел,7.382,actor
4,6966712,Хан Мин-ёп,33.0,NaN,None,1100409.0,Дьявольская радость,7.345,actor
9,14363,Пол Беттани,52.0,NaN,None,1203040.0,Ванда/Вижн,7.455,actor
10,14363,Пол Беттани,52.0,NaN,None,4422313.0,Очень британский скандал,6.533,actor
11,14363,Пол Беттани,52.0,NaN,None,1261534.0,"Что, если...?",6.928,actor


**Информация о персонах и их фильмах**

- **person_id**: Уникальный идентификатор персоны.
- **person_name**: Имя персоны.
- **age**: Возраст персоны.
- **countAwards**: Количество наград, полученных персоной.
- **death**: Дата смерти персоны (если применимо).
- **movie_id**: Уникальный идентификатор фильма, в котором участвовала персона.
- **movie_name**: Название фильма.
- **movie_rating**: Рейтинг фильма.
- **movie_profession**: Профессия персоны в данном фильме (например, актёр, режиссёр).

Данный DataFrame содержит информацию о различных персонах и фильмах, в которых они участвовали, а также детали об их профессии в этих фильмах.

In [17]:
df_one_person_one_movie.to_csv('data/one_person_one_movie.csv',**SAVE_DICT)

## Информация о фильмах, в которых участвовали персоны

In [18]:
ids_movies_all_persons = list(set(
    df_one_person_one_movie.movie_id.astype(int).astype(str)))

print('Всего уникальных id фильмов персон:', len(ids_movies_all_persons))

Всего уникальных id фильмов персон: 60717


In [ ]:
# Делим список из всех id фильмов на части.
# Каждая часть содержит 10000 id, чтобы оптимизировать кол-во запросов к API.

# Также при сборе фильмов персон, используем фильтр:
# кол-во оценок на Кинопоиске не менее 2000 - это прописано в config_params.py
ids = split_list(ids_movies_all_persons, 10000)

for i, chunk in enumerate(ids):
    """
    Для каждого чанка из 10000 id мы делаем запросы к API.
    Мы используем до 1000 id за один запрос. Точное количество запросов зависит
    от того, сколько страниц данных возвращает API для данного набора id.
    """
    
    json_info = fetcher.get_json_docs(
        API_URL_MOVIES, params_movies_additional, 'id', chunk, 1000)

    # Сохраняем данные для текущего чанка в отдельный JSON-файл.
    save_json(f'data_json/movies_all_persons_PARTS/json_ids_{i}', json_info)

    # Даём паузу между итерациями, чтобы не перегружать сервер и избежать блокировок.
    time.sleep(50)

# Объединяем все JSON-файлы в один общий список.
path = 'data_json/movies_all_persons_PARTS/*.json'
files = glob.glob(path)
json_movies_all_persons = [open_json(file) for file in files]

In [38]:
# Сохраняем полный json для фильмов персон 
save_json('data_json/movies_all_persons.json', json_movies_all_persons)

In [21]:
# Преобразуем всю полученную информацю в DataFrame
df_movies_all_persons = pd.DataFrame(get_movies(json_movies_all_persons))

print(df_movies_all_persons.shape)
df_movies_all_persons[:3]

(18686, 44)


,movie_id,movie_name,year,votes_kp,votes_imdb,votes_filmCritics,votes_await,rating_kp,rating_imdb,rating_filmCritics,...,fees_world,fees_world_currency,fees_usa,fees_usa_currency,fees_russia,fees_russia_currency,budget,budget_currency,videos_trailers_number,videos_teasers_number
0,8408,Гарри Поттер и Кубок огня,2005,509399,649477,255,3,7.946,7.7,7.4,...,895921036,$,290013036,$,7837694,$,150000000,$,5.0,0.0
1,387556,Хатико: Самый верный друг,2008,494048,296214,28,516,8.350,8.1,5.9,...,46184345,$,NaN,NaN,2046840,$,16000000,$,3.0,0.0
2,426004,Мальчишник в Вегасе,2009,368415,810907,240,709,7.864,7.7,6.8,...,467483912,$,277322503,$,5260019,$,35000000,$,3.0,0.0


Колонки в DataFrame с фильмами персон те же, что и в главной таблице с фильмами, которые были собраны при изначальном поиске фильмов на Кинопоиске

In [23]:
df_movies_all_persons.to_csv('data/movies_all_persons.csv', **SAVE_DICT)

# Обоснование решений относительно использования данных:

1. **Приквелы и сиквелы фильмов:** 
    - Некоторые фильмы, помеченные как приквелы или сиквелы, на самом деле оказались просто "похожими" на сайте КиноПоиск. 
    - В связи с этой неточностью, было решено не использовать эту информацию.


2. **Награды фильмов и персон:** 
    - Очень маленький процент фильмов и персон имеет награды.
    - Предоставленный API не возвращает полную информацию о наградах.
    - Изначальный анализ показал, что наличие наград не всегда коррелирует с коммерческим успехом фильма.
    - В связи с этими причинами, было решено не использовать информацию о наградах.


3. **Даты премьер фильмов:** 
    - У одного фильма может быть несколько дат премьеры, что создает путаницу.
    - Иногда отсутствует информация о локации премьеры (например, фестиваль или город).
    - Из-за этой неоднозначности и неполноты данных, было решено отказаться от использования дат премьер.